In [1]:
# Set me to the location of this directory...
%cd /Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/

/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban


In [2]:
import single_player_mcts as mc
import game_model as gm
import time
import json
import sqlite3
import datetime
import pandas


input_file_prefix = 'resources/from_class/'  # We will report on the input files below.
input_files = [input_file_prefix + f for f in ['sokoban00.txt', 'sokoban02.txt', 'sokoban03.txt', 'sokoban01.txt']]
output_db = 'out/results.db'

default_mcts_params = {
    'heuristic_f': gm.Evaluation.heuristic_2,
    'simulation_bound': 300,
    'exploration_c': 0.5,
    'uncertainty_d': 50.0,
    'heuristic_correction_f': lambda h, root_h: h
}
default_iteration_count = 2000


# Connect to our results database, and create the tables we need.
output_conn = sqlite3.connect(output_db)
output_cur = output_conn.cursor()
try:
    output_cur.execute("""
        CREATE TABLE Experiment (
            trial_id TEXT PRIMARY KEY,
            input_file TEXT,
            experiment TEXT
        );
    """)
    output_cur.execute("""
        CREATE TABLE Parameters (
            trial_id TEXT PRIMARY KEY,
            heuristic_f TEXT,
            simulation_bound INT,
            exploration_c FLOAT,
            uncertainty_d FLOAT,
            FOREIGN KEY (trial_id) REFERENCES Experiment (trial_id)
        );
    """)
    output_cur.execute("""
        CREATE TABLE Results (
            trial_id TEXT PRIMARY KEY,
            runtime FLOAT,
            iterations INT,
            solution_length INT,
            FOREIGN KEY (trial_id) REFERENCES Experiment (trial_id)
        );
    """)
    output_conn.commit()
except sqlite3.Error:
    pass  # There is no 'IF NOT EXISTS' support, so just "ask for forgiveness" here...


def perform_trial(input_file: str, mcts_params, experiment: str):
    # Build our game.
    root_state = gm.State.build(input_file)
    root_node = mc.Node(root_state)
    mcts_params['root'] = root_node
    x = mc.MCTS(**mcts_params)

    # Attempt to find a solution.
    starting_time = time.time()
    solution = x.run(default_iteration_count)
    end_time = time.time()

    # Setup the experiment and parameter table...
    primary_key = str(datetime.datetime.now())
    output_cur.execute(
        """ INSERT INTO Experiment VALUES (?, ?, ?); """,
        (primary_key, input_file, experiment, )
    )
    output_cur.execute(
        """ INSERT INTO Parameters VALUES  (?, ?, ?, ?, ?) """,
        (primary_key, str(mcts_params['heuristic_f']), mcts_params['simulation_bound'], mcts_params['exploration_c'],
         mcts_params['uncertainty_d'], )
    )

    # Save our results.
    if solution is not None:
        output_conn.execute(
            """ INSERT INTO Results VALUES (?, ?, ?, ?); """,
            (primary_key, end_time - starting_time, solution.number_of_iterations, len(solution.move_sequence))
        )
    else:
        output_conn.execute(
            """ INSERT INTO Results VALUES (?, ?, ?, ?); """,
            (primary_key, end_time - starting_time, None, None, )
        )


print('Running experiments with the following files: ')
print(json.dumps(input_files, indent=2))

Running experiments with the following files: 
[
  "resources/from_class/sokoban00.txt",
  "resources/from_class/sokoban02.txt",
  "resources/from_class/sokoban03.txt",
  "resources/from_class/sokoban01.txt"
]


In [3]:
# Setup plotting + analysis.
import mpld3
mpld3.enable_notebook()

import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [9.5, 6]

# Heuristic Experiments Below

In [4]:
# Test which heuristic is best for our input files.
heuristic_functions = [gm.Evaluation.heuristic_1, gm.Evaluation.heuristic_2, gm.Evaluation.heuristic_3]
heuristic_corrections = [lambda h, root_h: h, lambda h, root_h: h, gm.Evaluation.heuristic_3_correction]
heuristic_labels = ['heuristic_1', 'heuristic_2', 'heuristic_3']


# Perform our experiment.
number_of_experiments = 10
for input_file in input_files:
    for f, c, label in zip(heuristic_functions, heuristic_corrections, heuristic_labels):
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['heuristic_f'] = f
            mcts_params['heuristic_correction_f'] = c
            perform_trial(input_file, mcts_params, 'heuristic_f')

output_conn.commit()
print('Heuristic experiments are finished!')

Heuristic experiments are finished!


In [19]:
heuristic_f_data = pandas.read_sql_query(f"""
    SELECT A.heuristic_f, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'heuristic_f'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(heuristic_functions)}
    ) AS A;
""", output_conn)

heuristic_f_data.groupby(['heuristic_f', 'input_file']) \
    .agg([np.mean, np.std, len])['runtime']


mean  \
heuristic_f                                      input_file                                      
<function Evaluation.heuristic_1 at 0x11047f0e0> resources/from_class/sokoban00.txt   0.000628   
                                                 resources/from_class/sokoban01.txt  18.299114   
                                                 resources/from_class/sokoban02.txt   0.003923   
                                                 resources/from_class/sokoban03.txt   0.217749   
<function Evaluation.heuristic_2 at 0x11047f170> resources/from_class/sokoban00.txt   0.000452   
                                                 resources/from_class/sokoban01.txt  14.337613   
                                                 resources/from_class/sokoban02.txt   0.004053   
                                                 resources/from_class/sokoban03.txt   0.220337   
<function Evaluation.heuristic_3 at 0x11047f200> resources/from_class/sokoban00.txt   0.000376   
                                                 resources/from_class/sokoban01.txt  21.635429   
                                                 resources/from_class/sokoban02.txt   0.005800   
                                                 resources/from_class/sokoban03.txt   0.163557   

                                                                                           std  \
heuristic_f                                      input_file                                      
<function Evaluation.heuristic_1 at 0x11047f0e0> resources/from_class/sokoban00.txt   0.000306   
                                                 resources/from_class/sokoban01.txt  15.181983   
                                                 resources/from_class/sokoban02.txt   0.003141   
                                                 resources/from_class/sokoban03.txt   0.322966   
<function Evaluation.heuristic_2 at 0x11047f170> resources/from_class/sokoban00.txt   0.000086   
                                                 resources/from_class/sokoban01.txt  14.229313   
                                                 resources/from_class/sokoban02.txt   0.002902   
                                                 resources/from_class/sokoban03.txt   0.189857   
<function Evaluation.heuristic_3 at 0x11047f200> resources/from_class/sokoban00.txt   0.000008   
                                                 resources/from_class/sokoban01.txt  23.774159   
                                                 resources/from_class/sokoban02.txt   0.003846   
                                                 resources/from_class/sokoban03.txt   0.366087   

                                                                                      len  
heuristic_f                                      input_file                                
<function Evaluation.heuristic_1 at 0x11047f0e0> resources/from_class/sokoban00.txt  10.0  
                                                 resources/from_class/sokoban01.txt  10.0  
                                                 resources/from_class/sokoban02.txt  10.0  
                                                 resources/from_class/sokoban03.txt  10.0  
<function Evaluation.heuristic_2 at 0x11047f170> resources/from_class/sokoban00.txt  10.0  
                                                 resources/from_class/sokoban01.txt  10.0  
                                                 resources/from_class/sokoban02.txt  10.0  
                                                 resources/from_class/sokoban03.txt  10.0  
<function Evaluation.heuristic_3 at 0x11047f200> resources/from_class/sokoban00.txt  10.0  
                                                 resources/from_class/sokoban01.txt  10.0  
                                                 resources/from_class/sokoban02.txt  10.0  
                                                 resources/from_class/sokoban03.txt  10.0

# Simulation Bound Experiments Below

In [6]:
# Perform our experiment.
simulation_bounds = [20, 50, 100, 200, 500, 1000]
number_of_experiments = 10
for input_file in input_files:
    for b in simulation_bounds:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['simulation_bound'] = b
            perform_trial(input_file, mcts_params, 'simulation_bound')

output_conn.commit()
print('Simulation bound experiments are finished!')

Simulation bound experiments are finished!


In [20]:
simulation_bound_data = pandas.read_sql_query(f"""
    SELECT A.simulation_bound, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'simulation_bound'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(simulation_bounds)}
    ) AS A;
""", output_conn)

simulation_bound_data.groupby(['simulation_bound', 'input_file']) \
    .agg([np.mean, np.std, len])['runtime']

mean        std  \
simulation_bound input_file                                                 
20               resources/from_class/sokoban00.txt   0.000503   0.000167   
                 resources/from_class/sokoban01.txt  23.625373   0.178222   
                 resources/from_class/sokoban02.txt   0.004228   0.004475   
                 resources/from_class/sokoban03.txt   0.226353   0.190103   
50               resources/from_class/sokoban00.txt   0.000479   0.000157   
                 resources/from_class/sokoban01.txt  39.449378   0.695385   
                 resources/from_class/sokoban02.txt   0.004301   0.003313   
                 resources/from_class/sokoban03.txt   0.142006   0.072863   
100              resources/from_class/sokoban00.txt   0.000371   0.000024   
                 resources/from_class/sokoban01.txt  41.431558  20.082037   
                 resources/from_class/sokoban02.txt   0.004302   0.002683   
                 resources/from_class/sokoban03.txt   0.166351   0.149094   
200              resources/from_class/sokoban00.txt   0.000347   0.000001   
                 resources/from_class/sokoban01.txt  34.006372  23.563401   
                 resources/from_class/sokoban02.txt   0.004377   0.004000   
                 resources/from_class/sokoban03.txt   0.241418   0.316818   
500              resources/from_class/sokoban00.txt   0.000348   0.000002   
                 resources/from_class/sokoban01.txt  12.605563   8.024157   
                 resources/from_class/sokoban02.txt   0.003075   0.002321   
                 resources/from_class/sokoban03.txt   0.171187   0.265404   
1000             resources/from_class/sokoban00.txt   0.000350   0.000006   
                 resources/from_class/sokoban01.txt   9.665399  10.031467   
                 resources/from_class/sokoban02.txt   0.002845   0.002607   
                 resources/from_class/sokoban03.txt   0.092368   0.088290   

                                                      len  
simulation_bound input_file                                
20               resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0  
50               resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0  
100              resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0  
200              resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0  
500              resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0  
1000             resources/from_class/sokoban00.txt  10.0  
                 resources/from_class/sokoban01.txt  10.0  
                 resources/from_class/sokoban02.txt  10.0  
                 resources/from_class/sokoban03.txt  10.0

# Exploration C Experiments Below

In [8]:
# Perform our experiment.
exploration_c_terms = [0.25, 0.5, 0.75]
number_of_experiments = 10
for input_file in input_files:
    for c in exploration_c_terms:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['exploration_c'] = c
            perform_trial(input_file, mcts_params, 'exploration_c')

output_conn.commit()
print('Exploration C experiments are finished!')


Exploration C experiments are finished!


In [22]:
exploration_c_data = pandas.read_sql_query(f"""
    SELECT A.exploration_c, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'exploration_c'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(exploration_c_terms)}
    ) AS A;
""", output_conn)

exploration_c_data.groupby(['exploration_c', 'input_file']) \
    .agg([np.mean, np.std, len])['runtime']

mean        std   len
exploration_c input_file                                                    
0.25          resources/from_class/sokoban00.txt   0.000530   0.000455  10.0
              resources/from_class/sokoban01.txt  17.791537  18.859195  10.0
              resources/from_class/sokoban02.txt   0.005369   0.004921  10.0
              resources/from_class/sokoban03.txt   0.221991   0.163782  10.0
0.50          resources/from_class/sokoban00.txt   0.000362   0.000030  10.0
              resources/from_class/sokoban01.txt  13.008547  10.514809  10.0
              resources/from_class/sokoban02.txt   0.003147   0.002631  10.0
              resources/from_class/sokoban03.txt   0.137189   0.117847  10.0
0.75          resources/from_class/sokoban00.txt   0.000364   0.000028  10.0
              resources/from_class/sokoban01.txt  14.745970  11.689095  10.0
              resources/from_class/sokoban02.txt   0.005699   0.003772  10.0
              resources/from_class/sokoban03.txt   0.111956   0.126079  10.0

# Uncertainty D Experiments Below

In [10]:
# Perform our experiment.
uncertainty_d_terms = [25, 50, 100, 150]
number_of_experiments = 10
for input_file in input_files:
    for d in uncertainty_d_terms:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['uncertainty_d'] = d
            perform_trial(input_file, mcts_params, 'uncertainty_d')

output_conn.commit()
print('Uncertainty D experiments are finished!')

Uncertainty D experiments are finished!


In [23]:
uncertainty_d_data = pandas.read_sql_query(f"""
    SELECT A.uncertainty_d, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'uncertainty_d'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(uncertainty_d_terms)}
    ) AS A;
""", output_conn)

uncertainty_d_data.groupby(['uncertainty_d', 'input_file']) \
    .agg([np.mean, np.std, len])['runtime']

mean        std   len
uncertainty_d input_file                                                    
25.0          resources/from_class/sokoban00.txt   0.000400   0.000100  10.0
              resources/from_class/sokoban01.txt  17.503736  20.016591  10.0
              resources/from_class/sokoban02.txt   0.006357   0.005289  10.0
              resources/from_class/sokoban03.txt   0.158722   0.182720  10.0
50.0          resources/from_class/sokoban00.txt   0.000351   0.000002  10.0
              resources/from_class/sokoban01.txt  10.084509   8.757151  10.0
              resources/from_class/sokoban02.txt   0.004339   0.003972  10.0
              resources/from_class/sokoban03.txt   0.209560   0.213034  10.0
100.0         resources/from_class/sokoban00.txt   0.000356   0.000017  10.0
              resources/from_class/sokoban01.txt  21.238308  22.895188  10.0
              resources/from_class/sokoban02.txt   0.003443   0.004118  10.0
              resources/from_class/sokoban03.txt   0.160631   0.158726  10.0
150.0         resources/from_class/sokoban00.txt   0.000366   0.000019  10.0
              resources/from_class/sokoban01.txt  14.423557  10.995584  10.0
              resources/from_class/sokoban02.txt   0.003705   0.002666  10.0
              resources/from_class/sokoban03.txt   0.273240   0.311568  10.0